In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pickle

from iminuit import Minuit
from iminuit.cost import LeastSquares

import qubic.lib.Calibration.Qfiber as ft

plt.style.use('seaborn-v0_8-whitegrid')
 
%matplotlib inline

In [ ]:
# If True, allow plots for debug
DEBUG = False

In [ ]:
# Distances during experiment
distance_base_antenna1 = np.array([10, 10, 10, 10, 10, 10, 10, 8, 8, 8, 8, 8, 8, 8, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 3, 3, 3, 3, 2, 2, 2, 2, 1, 1]) * 0.60
distance_antenna1_antenna2 = np.array([7, 6, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 6, 7, 6, 5, 4, 3, 2, 1, 1, 2, 3, 4, 5, 6, 5, 4, 3, 2, 1, 1, 2, 3, 4, 2, 3, np.sqrt(2), np.sqrt(3), 2, 1]) * 0.60

if distance_base_antenna1.size != distance_antenna1_antenna2.size:
    print(distance_base_antenna1.size, distance_antenna1_antenna2.size)
    raise ValueError("The two arrays must have the same size")

In [ ]:
print("Number of configurations: ", distance_base_antenna1.size)

# Import data

In [ ]:
dict_data = pickle.load(open('GPS_noise_analysis.pkl', 'rb'))

data_names = np.array(['rpN', 'rpE', 'rpD', 'roll', 'yaw'])
names = ['rpN', 'rpE', 'rpD', 'roll', 'yaw']

distance_base_antenna2 = np.zeros(distance_base_antenna1.size)
for i in range(distance_base_antenna1.size):
    distance_base_antenna2[i] = np.sqrt((np.mean(np.array(dict_data['rpN']["clean_data"][i]))**2) + (np.mean(np.array(dict_data['rpE']["clean_data"][i]))**2) + (np.mean(np.array(dict_data['rpD']["clean_data"][i]))**2))

In [ ]:
plt.plot(distance_base_antenna2, label='distance base antenna2', color='black', linestyle='--')
plt.plot(distance_base_antenna1, label='distance base antenna1', color='red', linestyle='--')
plt.plot(distance_antenna1_antenna2, label='distance antenna1 antenna2', color='blue', linestyle='--')

plt.xlabel('Configuration')
plt.ylabel('Distance [m]')
plt.title('Distances during experiment')
plt.legend()

In [ ]:
print(dict_data.keys())
print(dict_data['rpN'].keys())
print(dict_data['rpN']['clean_data'].__len__())
print(dict_data['rpN']['clean_data'][0].size)

# Sandard Deviation

In [ ]:
std_gps = np.zeros((5, len(dict_data['rpN']['clean_data'])))

for idata in range(5):
    for jindex in range(len(dict_data['rpN']['clean_data'])):
        std_gps[idata, jindex] = np.std(dict_data[data_names[idata]]['clean_data'][jindex])
        
std_gps[:2] *= 100 # Convert to cm
std_gps[2:] = np.degrees(std_gps[2:]) # Convert to deg

In [ ]:
fig, ax1 = plt.subplots(dpi=300)

ax2 = ax1.twinx()

colors1 = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Blue, Orange, Green
colors2 = ['#d62728', '#9467bd']  # Red, Purple

for i in range(5):
    if i < 3:
        ax1.plot(std_gps[i], label=data_names[i], color=colors1[i])
    else:
        ax2.plot(std_gps[i], label=data_names[i], color=colors2[i-3], linestyle='--')
ax1.set_ylabel('Standard deviation [cm]')
ax2.set_ylabel('Standard deviation [deg]')
ax1.set_xlabel('Configuration index')
ax1.set_title('Standard deviation of the GPS data')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.tight_layout()

In [ ]:
unique_distances = np.unique(distance_base_antenna1)
colors = cm.jet(np.linspace(0, 1, len(unique_distances)))
data_name = 'rpN'
index_data = names.index(data_name)

fig, ax = plt.subplots(1, 1, figsize=(10, 8), sharex=True)
for i, dist in enumerate(unique_distances):
    mask = distance_base_antenna1 == dist
    ax.scatter(distance_antenna1_antenna2[mask], std_gps[index_data, mask], label=f'{dist:.1f}', color=colors[i], marker="x", s=50)

ax.set_ylabel("Standard Deviation (deg)")
ax.set_xlabel('Distance Antenna 1 to Antenna 2 (m)')

# Create a single legend outside the plot
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc='upper left', bbox_to_anchor=(1, 1), title='Distance Base to Antenna 1 (m)')

In [ ]:
data_name = 'rpN'
index_data = names.index(data_name)

plt.figure(dpi=300)

if data_name == 'roll' or data_name == 'yaw':
    unique_distances = np.unique(distance_base_antenna1)
    colors = cm.jet(np.linspace(0, 1, len(unique_distances)))

    fig, ax = plt.subplots(1, 1, figsize=(10, 8), sharex=True)
    for i, dist in enumerate(unique_distances):
        mask = distance_base_antenna1 == dist
        ax.scatter(distance_antenna1_antenna2[mask], std_gps[index_data, mask], label=f'{dist:.1f}', color=colors[i], marker="x", s=50)


    ax.set_ylabel("Standard Deviation (deg)")
    ax.set_xlabel('Distance Antenna 1 to Antenna 2 (m)')

    # Create a single legend outside the plot
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper left', bbox_to_anchor=(1, 1), title='Distance Base to Antenna 1 (m)')
    
else:
    unique_distances = np.unique(distance_antenna1_antenna2)
    colors = cm.jet(np.linspace(0, 1, len(unique_distances)))

    fig, ax = plt.subplots(1, 1, figsize=(10, 8), sharex=True)
    for i, dist in enumerate(unique_distances):
        mask = distance_antenna1_antenna2 == dist
        ax.scatter(distance_base_antenna1[mask], std_gps[index_data, mask], label=f'{dist:.1f}', color=colors[i], marker="x", s=50)


    ax.set_ylabel("Standard Deviation (m)")
    ax.set_xlabel('Distance Base to Antenna 1 (m)')

    # Create a single legend outside the plot
    handles, labels = ax.get_legend_handles_labels()
    fig.legend(handles, labels, loc='upper left', bbox_to_anchor=(1, 1), title='Distance Antenna 1 to Antenna 2 (m)')

fig.suptitle(f'Standard deviation - {data_name} data')
plt.tight_layout(rect=[0, 0, 1.05, 1])  # Adjust layout to make space for legend
plt.show()

# Noise Power Spectrum

## Def useful functions

In [ ]:
timestep = 0.125
print("Timestep : ", timestep, "s.")

def get_ps(array):
    """Function to compute the power spectrum of a given array.

    Parameters
    ----------
    array : array_like
        array containing the data to compute the power spectrum of.

    Returns
    -------
    power_spectrum : array_like
        array containing the power spectrum of the input array.
    freq: array_like
        array containing the frequency of the power spectrum.
    """
    N = array.size
    return np.abs(np.fft.rfft(array))**2, np.fft.rfftfreq(N, d=timestep/2)

In [ ]:
power_spectrum, freq = get_ps(dict_data['rpN']['clean_data'][0])
power_spectrum, freq = power_spectrum[1:], freq[1:]

#plt.figure(dpi=200)
plt.plot(freq, power_spectrum)
plt.xlabel('Frequency (Hz)')
plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
plt.xscale('log')
plt.yscale('log')
plt.title('Noise Power Spectrum - North - Index = 0')

del power_spectrum, freq

In [ ]:
def noise_model(x, A_white, f_knee, alpha):
    return A_white**2 * (1 + np.abs(f_knee/x)**alpha)

### Fit using my Gaussian LogLikelihood function with minuit

## Fit 1/f + white with minuit

In [ ]:
def nll_exp(A, f_knee, alpha, f, ps):
    P_model = noise_model(f, A, f_knee, alpha)
    return np.sum(np.log(P_model) + ps / P_model)

def nll_gauss(A, f_knee, alpha, freq, ps, sigma):
    P_model = noise_model(freq, A, f_knee, alpha)
    return np.sum(0.5 * ((ps - P_model) / sigma)**2 + 0.5 * np.log(2 * np.pi * sigma**2))

def fit_minuit_ll(data, nbins=300, plot=False, data_name=None, index=None):
    ps, freq = get_ps(data)
    ps, freq = ps[1:], freq[1:]
    
    binned_freq, binned_ps, _, binned_ps_error, _ = ft.profile(freq, ps, nbins=nbins, plot=False)
    
    def nll_wrapper(A, f_knee, alpha):
       return nll_gauss(A, f_knee, alpha, binned_freq, binned_ps, sigma=binned_ps_error)

    least_squares = LeastSquares(binned_freq, binned_ps, binned_ps_error, model = noise_model)
    m = Minuit(least_squares, A_white=0.1, f_knee=1, alpha=1)
    m.limits['A_white'] = (0, None)
    m.limits['f_knee'] = (0, None)
    m.limits['alpha'] = (0, None)

    m.migrad()
    m.hesse()
    
    if plot:
        #plt.figure(dpi=300)
        plt.plot(freq, ps, label="data")
        plt.plot(freq, noise_model(freq, *m.values), 'r', label="fit")

        # display legend with some fit info
        fit_info = [
            f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
        ]
        for p, v, e in zip(m.parameters, m.values, m.errors):
            fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")
        plt.title(f'Fit on Noise Power Spectrum - {data_name} - Index {index}')
        plt.legend(title="\n".join(fit_info), frameon=False)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
        plt.xscale("log")
        plt.yscale("log")
        plt.show()
        
    return m.values, m.errors, m.fmin.reduced_chi2

In [ ]:
params_values_loglike, params_errors_loglike, reduced_chi2_loglike = [], [], []
names = ['rpN', 'rpE', 'rpD', 'roll', 'yaw']
label = [r'$A_{white}$', r'$f_{knee}$', r'$\alpha$']

# Fit for each instrumental configurations
for idata in range(len(names)):
    values, errors, chi2_dof= [], [], []
    for index in range(len(dict_data['rpN']['clean_data'])):
        if index == 19:
            plot = True
        else:
            plot = False
        val, err, red_chi2 = fit_minuit_ll(dict_data[names[idata]]['clean_data'][index], nbins=int(dict_data[names[idata]]['clean_data'][index].size/11), plot=plot, data_name=names[idata], index=index)
        values.append(val)
        errors.append(err)
        chi2_dof.append(red_chi2)
    params_values_loglike.append(values)
    params_errors_loglike.append(errors)
    reduced_chi2_loglike.append(chi2_dof)
    
print(len(params_values_loglike))
print(len(params_values_loglike[0]))
print(len(params_values_loglike[0][0]))

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(15, 10), sharey='row', sharex=True) #, dpi=300)

n_datasets = len(names)
n_points   = len(params_values_loglike[0])  
for idata in range(n_datasets):
    for iparam in range(3):
        values = [params_values_loglike[idata][i][iparam] for i in range(n_points)]
        errors = [params_errors_loglike[idata][i][iparam] for i in range(n_points)]
        x = np.arange(n_points)
        
        ax[iparam, idata].errorbar(x, values, yerr=errors,
                                   fmt='o', color='black', capsize=3)
        
        mean_val = np.mean(values)
        std_val = np.std(values)
        ax[iparam, idata].axhline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean = {mean_val:.2f} | Std = {std_val:.2f}')
        ax[iparam, idata].legend(fontsize=8)

for iparam in range(3):
    ax[iparam, 0].set_ylabel(label[iparam], fontsize=12)

for idata in range(n_datasets):
    ax[0, idata].set_title(names[idata], fontsize=14)
    ax[1, idata].set_ylim(-1, 20)

for idata in range(n_datasets):
    ax[2, idata].set_xlabel('Instrumental Index', fontsize=12)
fig.suptitle(r'GPS noise analysis - logLikelihood - $P_{noise}(f) = A_{white}^2 (1 + |f_{knee}/f|^{\alpha})$', fontsize=16)
plt.tight_layout()
plt.savefig('GPS noise analysis - over f + White.png')
plt.show()

In [ ]:
fig = make_subplots(
    rows=3, cols=5, shared_xaxes=True, shared_yaxes=True,
    subplot_titles=names,
    vertical_spacing=0.05,   # decreased spacing for larger plots
    horizontal_spacing=0.03, # decreased spacing for larger plots
    row_heights=[0.1, 0.1, 0.1],
    column_widths=[0.1, 0.1, 0.1, 0.1, 0.1]
)

n_datasets = len(names)
n_points = len(params_values_loglike[0])

for idata in range(n_datasets):
    for iparam in range(3):
        values = [params_values_loglike[idata][i][iparam] for i in range(n_points)]
        errors = [params_errors_loglike[idata][i][iparam] for i in range(n_points)]
        x = np.arange(n_points)
        
        mean_val = np.mean(values)
        std_val = np.std(values)
        
        # Add error bars
        fig.add_trace(
            go.Scatter(
                x=x,
                y=values,
                error_y=dict(type='data', array=errors, visible=True),
                mode='markers',
                marker=dict(color="black"),
                name=names[idata],
                showlegend=False
            ),
            row=iparam+1, col=idata+1
        )
        
        # Add mean line
        fig.add_trace(
            go.Scatter(
                x=[0, n_points-1],
                y=[mean_val, mean_val],
                mode='lines',
                line=dict(color='red', dash='dash'),
                name=f'Mean = {mean_val:.2f} | Std = {std_val:.2f}',
                showlegend=False
            ),
            row=iparam+1, col=idata+1
        )

# Update layout
fig.update_layout(
    height=1000,
    width=1800,
    title_text=r'GPS noise analysis - logLikelihood - $P_{noise}(f) = A_{white}^2 (1 + |f_{knee}/f|^{\alpha})$',
    showlegend=False
)

# Update y-axis labels
for iparam in range(3):
    fig.update_yaxes(title_text=label[iparam], row=iparam+1, col=1)

# Update x-axis labels
for idata in range(n_datasets):
    fig.update_xaxes(title_text='Instrumental Index', row=3, col=idata+1)

fig.show()

## Are the parameters scaled with the distance between antennas ?

In [ ]:
data_name = 'yaw'
index_data = names.index(data_name)
params_value = np.array(params_values_loglike, dtype=float)[index_data]

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 8), sharex=True)#, dpi=300)

unique_distances = np.unique(distance_base_antenna2)
colors = cm.jet(np.linspace(0, 1, len(unique_distances)))

for i, dist in enumerate(unique_distances):
    mask = distance_base_antenna2 == dist
    ax[0].scatter(distance_antenna1_antenna2[mask], params_value[mask, 0], label=f'{dist:.1f}', color=colors[i])
    ax[1].scatter(distance_antenna1_antenna2[mask], params_value[mask, 1], color=colors[i])
    ax[2].scatter(distance_antenna1_antenna2[mask], params_value[mask, 2], color=colors[i])

ax[0].set_ylabel(r'$A_{white}$')
ax[1].set_ylabel(r'$f_{knee}$')
ax[2].set_ylabel(r'$\alpha$')
ax[2].set_xlabel('Distance Antenna 1 to Antenna 2')

# Create a single legend outside the plot
handles, labels = ax[0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1), title='Distance Antenna 1 to Antenna 2')
fig.suptitle(f'Noise power spectrum parameters for different distances - {data_name} data')
plt.tight_layout(rect=[0, 0, 1.05, 1])  # Adjust layout to make space for legend
plt.show()

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(10, 8), sharex=True)#, dpi=300)

unique_distances = np.unique(distance_antenna1_antenna2)
colors = cm.jet(np.linspace(0, 1, len(unique_distances)))

for i, dist in enumerate(unique_distances):
    mask = distance_antenna1_antenna2 == dist
    ax[0].scatter(distance_base_antenna2[mask], params_value[mask, 0], label=f'{dist:.1f}', color=colors[i])
    ax[1].scatter(distance_base_antenna2[mask], params_value[mask, 1], color=colors[i])
    ax[2].scatter(distance_base_antenna2[mask], params_value[mask, 2], color=colors[i])

ax[0].set_ylabel(r'$A_{white}$')
ax[1].set_ylabel(r'$f_{knee}$')
ax[2].set_ylabel(r'$\alpha$')
ax[2].set_xlabel('Distance base to Antenna 2')

# Create a single legend outside the plot
handles, labels = ax[0].get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(by_label.values(), by_label.keys(), loc='upper left', bbox_to_anchor=(1, 1), title='Distance Antenna 1 to Antenna 2')

fig.suptitle(f'Noise power spectrum parameters for different distances - {data_name} data')
plt.tight_layout(rect=[0, 0, 1.05, 1])  # Adjust layout to make space for legend
plt.show()

In [ ]:
fig = make_subplots(
    rows=3, cols=1, shared_xaxes=True, 
    vertical_spacing=0.05,
    subplot_titles=(r'$A_{white}$', r'$f_{knee}$', r'$\alpha$')
)

# Select distances and colors based on data_name
if data_name in ['roll', 'yaw']:
    unique_distances = np.unique(distance_base_antenna2)
    title_dist = 'Distance Base to Antenna 2'
    x_data = distance_antenna1_antenna2
    x_label = 'Distance Antenna 1 to Antenna 2'
else:
    unique_distances = np.unique(distance_antenna1_antenna2)
    title_dist = 'Distance Antenna 1 to Antenna 2'
    x_data = distance_base_antenna2
    x_label = 'Distance Base to Antenna 2'

# Generate a color palette
colors = px.colors.sample_colorscale('Jet', len(unique_distances))

# Add scatter traces for each distance
for dist, color in zip(unique_distances, colors):
    mask = (distance_base_antenna2 if data_name in ['roll', 'yaw'] else distance_antenna1_antenna2) == dist
    for i in range(3):
        show_legend = (i == 0)
        fig.add_trace(
            go.Scatter(
                x=x_data[mask],
                y=params_value[mask, i],
                mode='markers',
                name=f'{dist:.1f}',
                marker=dict(color=color),
                showlegend=show_legend
            ),
            row=i+1, col=1
        )

# Update axes labels and ranges
fig.update_yaxes(title_text=r'$A_{white}$', row=1, col=1)
fig.update_yaxes(title_text=r'$f_{knee}$', row=2, col=1, range=[-1, 10])
fig.update_yaxes(title_text=r'$\alpha$', row=3, col=1)
fig.update_xaxes(title_text=x_label, row=3, col=1)

# Layout adjustments: legend, title, margins
fig.update_layout(
    title_text=f'Noise power spectrum parameters for different distances - {data_name} data',
    title_x=0.5,
    legend=dict(title=title_dist, x=1.02, y=1),
    margin=dict(l=50, r=200, t=80, b=50),
    height=800,
    width=900
)

fig.show()

In [ ]:
stop

## Fit 1/f + linear noise

$P(f) = A_{white}^2 (1 + k*f + (\frac{f_{knee}}{f})^{\alpha})$

In [ ]:
def advanced_noise_model(x, A_white, f_knee, alpha, k):
    return A_white**2 * (1 + k*x + (f_knee/x)**alpha)

In [ ]:
def nll_gauss(A, f_knee, alpha, k, freq, ps, sigma):
    P_model = advanced_noise_model(freq, A, f_knee, alpha, k)
    return np.sum(0.5 * ((ps - P_model) / sigma)**2 + 0.5 * np.log(2 * np.pi * sigma**2))

def fit_minuit_ll(data, nbins=300, plot=False, data_name=None, index=None):
    ps, freq = get_ps(data)
    ps, freq = ps[1:], freq[1:]
    
    binned_freq, binned_ps, _, binned_ps_error, _ = ft.profile(freq, ps, nbins=nbins, plot=False)
    
    def nll_wrapper(A, f_knee, alpha, k):
       return nll_gauss(A, f_knee, alpha, k, binned_freq, binned_ps, sigma=binned_ps_error)
   
    least_squares = LeastSquares(binned_freq, binned_ps, binned_ps_error, model = advanced_noise_model)

    m = Minuit(least_squares, A_white=0.1, f_knee=1, alpha=1, k=1)
    m.limits['A_white'] = (0, None)
    m.limits['f_knee'] = (0, None)
    m.limits['alpha'] = (0, None)
    m.limits['k'] = (0, None)

    m.migrad()
    m.hesse()
    
    if plot:
        #plt.figure(dpi=300)
        plt.plot(freq, ps, label="data")
        plt.plot(freq, advanced_noise_model(freq, *m.values), 'r', label="fit")

        # display legend with some fit info
        fit_info = [
            f"$\\chi^2$/$n_\\mathrm{{dof}}$ = {m.fval:.1f} / {m.ndof:.0f} = {m.fmin.reduced_chi2:.1f}",
        ]
        for p, v, e in zip(m.parameters, m.values, m.errors):
            fit_info.append(f"{p} = ${v:.3f} \\pm {e:.3f}$")
        plt.title(f'Fit on Noise Power Spectrum - {data_name} - Index {index}')
        plt.legend(title="\n".join(fit_info), frameon=False)
        plt.xlabel('Frequency (Hz)')
        plt.ylabel(r'Power Spectrum ($m^2/Hz$)')
        plt.xscale("log")
        plt.yscale("log")
        plt.show()
        
    return m.values, m.errors, m.fmin.reduced_chi2

In [ ]:
params_values_loglike, params_errors_loglike, reduced_chi2_loglike = [], [], []
names = ['rpN', 'rpE', 'rpD', 'roll', 'yaw']
label = [r'$A_{white}$', r'$f_{knee}$', r'$\alpha$', r'k']

# Fit for each instrumental configurations
for idata in range(len(names)):
    values, errors, chi2_dof= [], [], []
    for index in range(len(dict_data[names[idata]]['clean_data'])):
        if index == 6:
            plot = True
        else:
            plot = False
        val, err, red_chi2 = fit_minuit_ll(dict_data[names[idata]]['clean_data'][index], nbins=int(dict_data[names[idata]]['clean_data'][index].size/8), plot=plot, data_name=names[idata], index=index)
        values.append(val)
        errors.append(err)
        chi2_dof.append(red_chi2)
    params_values_loglike.append(values)
    params_errors_loglike.append(errors)
    reduced_chi2_loglike.append(chi2_dof)
    
print(len(params_values_loglike))
print(len(params_values_loglike[0]))
print(len(params_values_loglike[0][0]))

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
fig, ax = plt.subplots(len(label), 5, figsize=(15, 10), sharey='row', sharex=True)#, dpi=300)

n_datasets = len(names)
n_points   = len(params_values_loglike[0])  
for idata in range(n_datasets):
    for iparam in range(len(label)):
        values = [params_values_loglike[idata][i][iparam] for i in range(n_points)]
        errors = [params_errors_loglike[idata][i][iparam] for i in range(n_points)]
        x = np.arange(n_points)
        
        ax[iparam, idata].errorbar(x, values, yerr=errors,
                                   fmt='o', color='black', capsize=3)
        
        mean_val = np.mean(values)
        std_val = np.std(values)
        ax[iparam, idata].axhline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Mean = {mean_val:.2f} | Std = {std_val:.2f}')
        ax[iparam, idata].legend(fontsize=8)

for iparam in range(len(label)):
    ax[iparam, 0].set_ylabel(label[iparam], fontsize=12)

for idata in range(n_datasets):
    ax[0, idata].set_title(names[idata], fontsize=14)
    ax[0, idata].set_ylim(0, 0.1)
    ax[1, idata].set_ylim(-1, 250)
    ax[2, idata].set_ylim(0.5, 2.5)
    ax[3, idata].set_ylim(-1, 50)

for idata in range(n_datasets):
    ax[-1, idata].set_xlabel('Instrumental Index', fontsize=12)
fig.suptitle(r'GPS noise analysis - logLikelihood - $P_{noise}(f) = A_{white}^2 (1 + k*f + |f_{knee}/f|^{\alpha})$', fontsize=16)
plt.tight_layout()
plt.savefig('GPS noise analysis - over f + Linear.png')
plt.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=4, cols=5, shared_xaxes=True, shared_yaxes=True,
    subplot_titles=names,
    vertical_spacing=0.05,   # decreased spacing for larger plots
    horizontal_spacing=0.03, # decreased spacing for larger plots
    row_heights=[0.1, 0.1, 0.1, 0.1],
    column_widths=[0.1, 0.1, 0.1, 0.1, 0.1]
)

n_datasets = len(names)
n_points = len(params_values_loglike[0])

for idata in range(n_datasets):
    for iparam in range(4):
        values = [params_values_loglike[idata][i][iparam] for i in range(n_points)]
        errors = [params_errors_loglike[idata][i][iparam] for i in range(n_points)]
        x = np.arange(n_points)
        
        mean_val = np.mean(values)
        std_val = np.std(values)
        
        # Add error bars
        fig.add_trace(
            go.Scatter(
                x=x,
                y=values,
                error_y=dict(type='data', array=errors, visible=True),
                mode='markers',
                marker=dict(color="black"),
                name=names[idata],
                showlegend=False
            ),
            row=iparam+1, col=idata+1
        )
        
        # Add mean line
        fig.add_trace(
            go.Scatter(
                x=[0, n_points-1],
                y=[mean_val, mean_val],
                mode='lines',
                line=dict(color='red', dash='dash'),
                name=f'Mean = {mean_val:.2f} | Std = {std_val:.2f}',
                showlegend=False
            ),
            row=iparam+1, col=idata+1
        )

# Update layout
fig.update_layout(
    height=1000,
    width=1800,
    title_text=r'GPS noise analysis - logLikelihood - $P_{noise}(f) = A_{white}^2 (1 + k*f + |f_{knee}/f|^{\alpha})$',
    showlegend=False
)

# Update y-axis labels
for iparam in range(4):
    fig.update_yaxes(title_text=label[iparam], row=iparam+1, col=1)

# Update x-axis labels
for idata in range(n_datasets):
    fig.update_xaxes(title_text='Instrumental Index', row=4, col=idata+1)

fig.show()

In [ ]:
params_value = np.array(params_values_loglike, dtype=float)[index_data]
print(params_value.shape)

In [ ]:

unique_distances = np.unique(distance_base_antenna1)
colors = px.colors.sample_colorscale("jet", [i/(len(unique_distances)-1) for i in range(len(unique_distances))])

# Create a subplot figure with 2 rows and 1 column, sharing the x-axis.
fig = make_subplots(
    rows=4, cols=1, shared_xaxes=True,
    vertical_spacing=0.1,
)

# Loop over unique distances to add traces for both subplots.
for i, dist in enumerate(unique_distances):
    mask = distance_base_antenna1 == dist
    # First subplot: A_white values
    fig.add_trace(
        go.Scatter(
            x=distance_antenna1_antenna2[mask],
            y=params_value[mask, 0],
            mode='markers',
            name=f'{dist:.1f}',
            marker=dict(color=colors[i])
        ),
        row=1, col=1
    )
    # Second subplot: f_knee values; hide legend to avoid duplicates.
    fig.add_trace(
        go.Scatter(
            x=distance_antenna1_antenna2[mask],
            y=params_value[mask, 1],
            mode='markers',
            name=f'{dist:.1f}',
            marker=dict(color=colors[i]),
            showlegend=False
        ),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=distance_antenna1_antenna2[mask],
            y=params_value[mask, 2],
            mode='markers',
            name=f'{dist:.1f}',
            marker=dict(color=colors[i]),
            showlegend=False
        ),
        row=3, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=distance_antenna1_antenna2[mask],
            y=params_value[mask, 3],
            mode='markers',
            name=f'{dist:.1f}',
            marker=dict(color=colors[i]),
            showlegend=False
        ),
        row=4, col=1
    )

# Update axis labels and overall layout.
fig.update_xaxes(title_text='Distance Antenna 1 to Antenna 2', row=4, col=1)
fig.update_yaxes(title_text=label[0], row=1, col=1)
fig.update_yaxes(title_text=label[1], row=2, col=1)
fig.update_yaxes(title_text=label[2], row=3, col=1)
fig.update_yaxes(title_text=label[3], row=4, col=1)

fig.update_layout(
    title=f'Noise power spectrum parameters for different distances - {data_name} data',
    # Position the legend outside the plot on the right.
    legend=dict(
        title='Distance Base to Antenna 2',
        orientation='v',
        x=1.02,
        y=1,
        xanchor='left',
        yanchor='top'
    ),
    width=1200,
    height=800,
    margin=dict(r=150)  # Increase right margin to accommodate the legend.
)

fig.show()

In [ ]:
unique_distances = np.unique(distance_base_antenna1)
colors = cm.jet(np.linspace(0, 1, len(unique_distances)))

fig, ax = plt.subplots(4, 1, figsize=(10, 8), sharex=True)

for i, dist in enumerate(unique_distances):
    mask = distance_base_antenna1 == dist
    ax[0].scatter(distance_antenna1_antenna2[mask], params_value[mask, 0], label=f'{dist:.1f}', color=colors[i])
    ax[1].scatter(distance_antenna1_antenna2[mask], params_value[mask, 1], color=colors[i])
    ax[2].scatter(distance_antenna1_antenna2[mask], params_value[mask, 2], color=colors[i])
    ax[3].scatter(distance_antenna1_antenna2[mask], params_value[mask, 3], color=colors[i])

ax[0].set_ylabel(r'$A_{white}$')
ax[1].set_ylabel(r'$f_{knee}$')
ax[1].set_ylim(0, 100)
ax[2].set_ylabel(r'$\alpha$')
ax[3].set_ylabel(r'k')
ax[-1].set_xlabel('Distance Antenna 1 to Antenna 2')

# Create a single legend outside the plot
handles, labels = ax[0].get_legend_handles_labels()
fig.legend(handles, labels, loc='upper left', bbox_to_anchor=(1, 1), title='Distance Base to Antenna 2')

fig.suptitle(f'Noise power spectrum parameters for different distances - {data_name} data')
plt.tight_layout(rect=[0, 0, 1.05, 1])  # Adjust layout to make space for legend
plt.show()